In [2]:
# equations / theory from here
# https://www.datacamp.com/blog/predicting-soccer-world-cup-winners
# https://doi.org/10.1016/j.ijforecast.2009.10.002
# https://medium.com/purple-theory/what-is-elo-rating-c4eb7a9061e0

# probability of team a winning
def win_prob(rating_a, rating_b):
  diff = rating_b - rating_a
  return 1 / (1+10**(diff/400))

# calculate new elo of team a based on match result
def update_elo(old_elo, result, win_probability):
  # result is 0 for loss, 1 for win, and 0.5 for draw
  return old_elo + 50*(result-win_probability)

In [3]:
import pandas as pd

In [4]:
# data from here https://fixturedownload.com/results/fifa-world-cup-2022

from google.colab import files
uploaded = files.upload()

Saving fifa-world-cup-2022-UTC.csv to fifa-world-cup-2022-UTC.csv


In [9]:
#data = {'Country': ['Netherlands','Senegal','Ecuador','Qatar','England','USA','Iran','Wales','Argentina','Poland','Mexico','Saudi Arabia','France','Australia','Tunisia','Denmark','Japan','Spain','Germany','Costa Rica','Morocco','Croatia','Belgium','Canada','Brazil','Switzerland','Cameroon','Serbia','Portugal','South Korea','Uruguay','Ghana'],
#        'ELO': [1886, 1732, 1736, 1656, 1876, 1782, 1811, 1705, 1944, 1740, 1769, 1665, 1906, 1728, 1684, 1820, 1780, 1915, 1862, 1676, 1753, 1800, 1897, 1678, 2000, 1802, 1643, 1753, 1868, 1789, 1814, 1616]}
#initial_rankings = pd.DataFrame(data)
#rankings = pd.DataFrame(data)

In [17]:
initial_rankings = {'Netherlands':1886, 'Senegal':1732, 'Ecuador':1736, 'Qatar':1656, 'England':1876, 'USA':1782, 'Iran':1811, 'Wales':1705, 'Argentina':1944, 'Poland':1740, 'Mexico':1769, 'Saudi Arabia':1665, 'France':1906, 'Australia':1728, 'Tunisia':1684, 'Denmark':1820, 'Japan':1780, 'Spain':1915, 'Germany':1862, 'Costa Rica':1676, 'Morocco':1753, 'Croatia':1800, 'Belgium':1897, 'Canada':1678, 'Brazil':2000, 'Switzerland':1802, 'Cameroon':1643, 'Serbia':1753, 'Portugal':1868, 'Korea Republic':1789, 'Uruguay':1814, 'Ghana':1616}

In [12]:
rankings = {'Netherlands':1886, 'Senegal':1732, 'Ecuador':1736, 'Qatar':1656, 'England':1876, 'USA':1782, 'Iran':1811, 'Wales':1705, 'Argentina':1944, 'Poland':1740, 'Mexico':1769, 'Saudi Arabia':1665, 'France':1906, 'Australia':1728, 'Tunisia':1684, 'Denmark':1820, 'Japan':1780, 'Spain':1915, 'Germany':1862, 'Costa Rica':1676, 'Morocco':1753, 'Croatia':1800, 'Belgium':1897, 'Canada':1678, 'Brazil':2000, 'Switzerland':1802, 'Cameroon':1643, 'Serbia':1753, 'Portugal':1868, 'Korea Republic':1789, 'Uruguay':1814, 'Ghana':1616}

In [7]:
# set all rankings to 1500

for key in rankings:
  rankings[key] = 1500

In [13]:
matches = pd.read_csv('fifa-world-cup-2022-UTC.csv') # read csv file

matches.iloc[62,5] = 'Morocco' # add predicted teams
matches.iloc[63,5] = 'France'

matches['Home Goals'] = matches['Result'].str[0] # get goals for each team
matches['Away Goals'] = matches['Result'].str[4]

#matches_rankings = matches.replace({"Home Team": rankings}) # get rankings for each team
#matches_rankings = matches_rankings.replace({"Away Team": rankings})
#matches_rankings.rename(columns = {'Home Team':'Home Ranking', 'Away Team':'Away Ranking'}, inplace = True)
#matches_rankings = matches_rankings.loc[:,['Home Ranking','Away Ranking']]
#matches = pd.concat([matches, matches_rankings], axis=1)

In [14]:
matches

,Match Number,Round Number,Date,Location,Home Team,Away Team,Group,Result,Home Goals,Away Goals
0,1,1,20/11/2022 16:00,Al Bayt Stadium,Qatar,Ecuador,Group A,0 - 2,0,2
1,3,1,21/11/2022 13:00,Khalifa International Stadium,England,Iran,Group B,6 - 2,6,2
2,2,1,21/11/2022 16:00,Al Thumama Stadium,Senegal,Netherlands,Group A,0 - 2,0,2
3,4,1,21/11/2022 19:00,Ahmad Bin Ali Stadium,USA,Wales,Group B,1 - 1,1,1
4,8,1,22/11/2022 10:00,Lusail Stadium,Argentina,Saudi Arabia,Group C,1 - 2,1,2
...,...,...,...,...,...,...,...,...,...,...
59,59,Quarter Finals,10/12/2022 19:00,Al Bayt Stadium,England,France,NaN,1 - 2,1,2
60,61,Semi Finals,13/12/2022 19:00,Lusail Stadium,Argentina,Croatia,NaN,3 - 0,3,0
61,62,Semi Finals,14/12/2022 19:00,Al Bayt Stadium,France,Morocco,NaN,NaN,NaN,NaN
62,63,Finals,17/12/2022 15:00,Khalifa International Stadium,Croatia,Morocco,NaN,NaN,NaN,NaN


In [15]:
for index, row in matches.iterrows():
  #print(win_probability(row['Home Ranking'], row['Away Ranking']))
  #print(row['Away Ranking'])
  home_team = row['Home Team']
  away_team = row['Away Team']
  home_ranking = rankings[home_team]
  away_ranking = rankings[away_team]
  home_win = win_prob(home_ranking, away_ranking)
  away_win = win_prob(away_ranking, home_ranking)

  home_goals = row['Home Goals']
  away_goals = row['Away Goals']
  if home_goals > away_goals:
    home_result = 1
    away_result = 0
  if home_goals == away_goals:
    home_result = 0.5
    away_result = 0.5
  if home_goals < away_goals:
    home_result = 0
    away_result = 1
  home_new_ranking = update_elo(home_ranking, home_result, home_win)
  away_new_ranking = update_elo(away_ranking, away_result, away_win)
  rankings[home_team] = home_new_ranking
  rankings[away_team] = away_new_ranking
  print('{} ({:.0f}|{:.0%}) vs {} ({:.0f}|{:.0%})'.format(home_team, home_ranking, home_win, away_team, away_ranking, away_win))
  print('{} {} ({}|{:.0f}) vs {} {} ({}|{:.0f})'.format(home_team, home_goals, home_result, home_new_ranking, away_team, away_goals, away_result, away_new_ranking))

Qatar (1656|39%) vs Ecuador (1736|61%)
Qatar 0 (0|1637) vs Ecuador 2 (1|1755)
England (1876|59%) vs Iran (1811|41%)
England 6 (1|1896) vs Iran 2 (0|1791)
Senegal (1732|29%) vs Netherlands (1886|71%)
Senegal 0 (0|1717) vs Netherlands 2 (1|1901)
USA (1782|61%) vs Wales (1705|39%)
USA 1 (0.5|1777) vs Wales 1 (0.5|1710)
Argentina (1944|83%) vs Saudi Arabia (1665|17%)
Argentina 1 (0|1902) vs Saudi Arabia 2 (1|1707)
Denmark (1820|69%) vs Tunisia (1684|31%)
Denmark 0 (0.5|1811) vs Tunisia 0 (0.5|1693)
Mexico (1769|54%) vs Poland (1740|46%)
Mexico 0 (0.5|1767) vs Poland 0 (0.5|1742)
France (1906|74%) vs Australia (1728|26%)
France 4 (1|1919) vs Australia 1 (0|1715)
Morocco (1753|43%) vs Croatia (1800|57%)
Morocco 0 (0.5|1756) vs Croatia 0 (0.5|1797)
Germany (1862|62%) vs Japan (1780|38%)
Germany 1 (0|1831) vs Japan 2 (1|1811)
Spain (1915|80%) vs Costa Rica (1676|20%)
Spain 7 (1|1925) vs Costa Rica 0 (0|1666)
Belgium (1897|78%) vs Canada (1678|22%)
Belgium 1 (1|1908) vs Canada 0 (0|1667)
Switze

In [20]:
# calculate change in elo score
performance = {}
for key in rankings:
  performance[key] = rankings[key] - initial_rankings[key]

In [18]:
# who is now the best team?

for key in sorted(rankings, key=rankings.get, reverse=True):
  print("{} -> {:.0f} -> {:.0f}".format(key, initial_rankings[key], rankings[key]))

Argentina -> 1944 -> 1986
Brazil -> 2000 -> 1978
France -> 1906 -> 1930
Netherlands -> 1886 -> 1920
England -> 1876 -> 1891
Spain -> 1915 -> 1878
Belgium -> 1897 -> 1868
Portugal -> 1868 -> 1855
Germany -> 1862 -> 1853
Croatia -> 1800 -> 1837
Japan -> 1780 -> 1811
Uruguay -> 1814 -> 1807
Switzerland -> 1802 -> 1802
Morocco -> 1753 -> 1798
USA -> 1782 -> 1793
Iran -> 1811 -> 1783
Korea Republic -> 1789 -> 1776
Mexico -> 1769 -> 1771
Denmark -> 1820 -> 1764
Australia -> 1728 -> 1753
Senegal -> 1732 -> 1748
Ecuador -> 1736 -> 1738
Poland -> 1740 -> 1735
Serbia -> 1753 -> 1715
Tunisia -> 1684 -> 1711
Costa Rica -> 1676 -> 1685
Cameroon -> 1643 -> 1682
Wales -> 1705 -> 1679
Saudi Arabia -> 1665 -> 1664
Canada -> 1678 -> 1635
Ghana -> 1616 -> 1629
Qatar -> 1656 -> 1609


In [23]:
# who improved the most?

for key in sorted(performance, key=performance.get, reverse=True):
  print("{} -> {:.0f} -> {:.0f} | {:.0f}".format(key, initial_rankings[key], rankings[key], performance[key]))

Morocco -> 1753 -> 1798 | 45
Argentina -> 1944 -> 1986 | 42
Cameroon -> 1643 -> 1682 | 39
Croatia -> 1800 -> 1837 | 37
Netherlands -> 1886 -> 1920 | 34
Japan -> 1780 -> 1811 | 31
Tunisia -> 1684 -> 1711 | 27
Australia -> 1728 -> 1753 | 25
France -> 1906 -> 1930 | 24
Senegal -> 1732 -> 1748 | 16
England -> 1876 -> 1891 | 15
Ghana -> 1616 -> 1629 | 13
USA -> 1782 -> 1793 | 11
Costa Rica -> 1676 -> 1685 | 9
Mexico -> 1769 -> 1771 | 2
Ecuador -> 1736 -> 1738 | 2
Switzerland -> 1802 -> 1802 | -0
Saudi Arabia -> 1665 -> 1664 | -1
Poland -> 1740 -> 1735 | -5
Uruguay -> 1814 -> 1807 | -7
Germany -> 1862 -> 1853 | -9
Portugal -> 1868 -> 1855 | -13
Korea Republic -> 1789 -> 1776 | -13
Brazil -> 2000 -> 1978 | -22
Wales -> 1705 -> 1679 | -26
Iran -> 1811 -> 1783 | -28
Belgium -> 1897 -> 1868 | -29
Spain -> 1915 -> 1878 | -37
Serbia -> 1753 -> 1715 | -38
Canada -> 1678 -> 1635 | -43
Qatar -> 1656 -> 1609 | -47
Denmark -> 1820 -> 1764 | -56
